## Task 1 
#### Classifies road lanes as being occupied or not

In [1]:
# Importing libraries
import cv2
import os
import io
import glob
import csv
import numpy as np
from ultralytics import YOLO
model = YOLO("yolov8x.pt") # x means Huge size of YOLO
cv2.__version__

'4.7.0'

In [2]:
# Required files paths 
Train_Mode = False

if Train_Mode:
    images = r'D:\Uni_Buc\Uni_Buc_semester_2\Computer Vision\CV 2023\CV-2023-Project2\train\Task1\*.jpg'
    text_files = r'D:\Uni_Buc\Uni_Buc_semester_2\Computer Vision\CV 2023\CV-2023-Project2\train\Task1\*.txt'
else:
    images = r'D:\Uni_Buc\Uni_Buc_semester_2\Computer Vision\CV 2023\CV-2023-Project2\test\Task1\*.jpg'
    text_files = r'D:\Uni_Buc\Uni_Buc_semester_2\Computer Vision\CV 2023\CV-2023-Project2\test\Task1\*.txt'
    
images_paths = sorted(glob.glob(images), key=lambda i: (os.path.splitext(os.path.basename(i))[0]))
txt_paths = sorted(glob.glob(text_files), key=lambda i: (os.path.splitext(os.path.basename(i))[0]))

results_paths = r'D:\Uni_Buc\Uni_Buc_semester_2\Computer Vision\CV 2023\CV-2023-Project2\\Lara_Tomeh_507\Task1'
ground_truth_paths = r'D:\Uni_Buc\Uni_Buc_semester_2\Computer Vision\CV 2023\CV-2023-Project2\\train\Task1\ground-truth'

In [3]:
# Read the text files
def read_lanes():
    number_of_lanes = []
    lanes_ids = []
    for i, txt in enumerate(txt_paths):
        with open(txt) as f:
            total_lanes = int(f.readline().split(' ')[0])
            number_of_lanes.append(total_lanes)

            ids = []
            for _ in range(total_lanes):
                line = f.readline().split()
                lane_id = int(line[0])
                ids.append(lane_id)
            lanes_ids.append(ids)

    return number_of_lanes, lanes_ids

number_of_lanes, lanes_ids = read_lanes()

# Create the predicted lanes dictionary
predicted_lanes = {}
for i, image_path in enumerate(images, start=1):
    predicted_lanes[i] = []

In [4]:
def detect_lanes(images, number_of_lanes, lanes_ids, txt_paths, results_paths):
    
    predicted_lanes = {i+1: [] for i in range(len(images))}           # Initialize predicted_lanes dictionary
    
    # for loop to read all images in the path
    for i, image in enumerate(images):
        print(f'Detecting image {i+1}/{len(images)}')
        
        # Opjects detection using YOLO model
        objects = model.predict(image, classes=[1, 2, 3, 5, 6, 7, 8])  # Select which classes to predict
        detections = objects[0]
        
        # Define the polygonal ROI for each lane 
        # After several experiments, they were determined due to the close proximity of the paths to each other."
        roi_points_lane1 = np.array([(270, 463), (460, 440), (126, 133), (116,205)], np.int32)
        roi_points_lane2 = np.array([(460, 437), (588, 417), (101, 58), (89, 96)], np.int32)
#         roi_points_lane3 = np.array([(586, 417), (713, 398), (93, 2), (72, 24)], np.int32)
        roi_points_lane3 = np.array([(99, 55), (581, 414), (759, 391), (175, 41)], np.int32)
        roi_points_lane4u = np.array([(1145,359), (1913,279), (1896,204), (1113,264)], np.int32)
#         roi_points_lane4d = np.array([(1157,373), (1917,298), (1915,280), (1144,359)], np.int32)
        roi_points_lane4upup = np.array([(1843, 177), (1062, 182), (1068, 261), (1837, 235)], np.int32)
        roi_points_lane4 = np.array([(1912, 265), (1912, 338), (1194, 420), (1118, 365)], np.int32)
        roi_points_lane5 = np.array([(1913, 362), (1180, 438), (1161, 409), (1914, 320)], np.int32)
        roi_points_lane6 = np.array([(1913, 364), (1912, 402), (1228, 461), (1213, 434)], np.int32)
        roi_points_lane7 = np.array([(1161, 581), (1648, 874), (1917, 874), (1914, 676), (1442, 483)], np.int32)
        roi_points_lane8 = np.array([(1391, 880), (1006, 619), (1179, 595), (1650, 880)], np.int32)
        roi_points_lane9 = np.array([(1385, 876), (1000, 611), (795, 645), (1035, 877)], np.int32)

        # Initialize counters for cars in each lane
        car_count_lane1 = 0
        car_count_lane2 = 0
        car_count_lane3 = 0
        car_count_lane4 = 0
        car_count_lane5 = 0
        car_count_lane6 = 0
        car_count_lane7 = 0
        car_count_lane8 = 0
        car_count_lane9 = 0
        
        # Iterate through all detections
        for detection in detections:
            cords = detection.boxes.xyxy[0].tolist()
            xmin = cords[0]
            ymin = cords[1]
            xmax = cords[2]
            ymax = cords[3]
            conf = detection.boxes.conf
            centroid_x = int((xmin + xmax) / 2) # centre point x coordinates
            centroid_y = int((ymin + ymax) / 2) # centre point y coordinates

            # Define the points as tuples
            point1 = (xmin, ymin)
            point2 = (xmin, ymax)
            point3 = (xmax, ymax)
            point4 = (xmax, ymin)

            # Check if the points are inside each ROI and assign the car to the corresponding lane
#         ----------------------------LANE 1---------------------------------------------
            if (
                cv2.pointPolygonTest(roi_points_lane1, point1, False) > 0
                and cv2.pointPolygonTest(roi_points_lane1, point2, False) > 0
            ):
                cv2.rectangle(image, (int(xmin), int(ymin)), (int(xmax), int(ymax)), (0, 255, 0), 2)
                cv2.circle(image, (centroid_x, centroid_y), 5, (0, 255, 0), -1)
                car_count_lane1 += 1
        #         --------------------------LANE 2-----------------------------------------------
            if (
                cv2.pointPolygonTest(roi_points_lane2, point1, False) > 0
                and cv2.pointPolygonTest(roi_points_lane2, point3, False) > 0
                and cv2.pointPolygonTest(roi_points_lane1, point2, False) > 0
                and cv2.pointPolygonTest(roi_points_lane3, point4, False) > 0
            ):
                cv2.rectangle(image, (int(xmin), int(ymin)), (int(xmax), int(ymax)), (255, 0, 0), 2)
                cv2.circle(image, (centroid_x, centroid_y), 5, (255, 0, 0), -1)
                car_count_lane2 += 1
        #         -------------------------------------------------------------------------
            if (
                cv2.pointPolygonTest(roi_points_lane2, point1, False) > 0
                and cv2.pointPolygonTest(roi_points_lane2, point3, False) > 0
                and cv2.pointPolygonTest(roi_points_lane2, point2, False) > 0
                and cv2.pointPolygonTest(roi_points_lane3, point4, False) > 0
            ):
                cv2.rectangle(image, (int(xmin), int(ymin)), (int(xmax), int(ymax)), (255, 0, 0), 2)
                cv2.circle(image, (centroid_x, centroid_y), 5, (255, 0, 0), -1)
                car_count_lane2 += 1
        #         -------------------------------------------------------------------------
            if (
                cv2.pointPolygonTest(roi_points_lane3, point1, False) > 0
                and cv2.pointPolygonTest(roi_points_lane2, point3, False) > 0
                and cv2.pointPolygonTest(roi_points_lane1, point2, False) > 0
            ):
                cv2.rectangle(image, (int(xmin), int(ymin)), (int(xmax), int(ymax)), (255, 0, 0), 2)
                cv2.circle(image, (centroid_x, centroid_y), 5, (255, 0, 0), -1)
                car_count_lane2 += 1
        #         -------------------------------------------------------------------------
            if (
                cv2.pointPolygonTest(roi_points_lane2, point1, False) > 0
                and cv2.pointPolygonTest(roi_points_lane2, point3, False) > 0
            ):
                cv2.rectangle(image, (int(xmin), int(ymin)), (int(xmax), int(ymax)), (255, 0, 0), 2)
                cv2.circle(image, (centroid_x, centroid_y), 5, (255, 0, 0), -1)
                car_count_lane2 += 1
        #         ------------------LANE 3-------------------------------------------------------
            if (
                cv2.pointPolygonTest(roi_points_lane3, point1, False) > 0
                and cv2.pointPolygonTest(roi_points_lane3, point2, False) > 0
            ):
                cv2.rectangle(image, (int(xmin), int(ymin)), (int(xmax), int(ymax)), (0, 0, 255), 2)
                cv2.circle(image, (centroid_x, centroid_y), 5, (0, 0, 255), -1)
                car_count_lane3 += 1
        #         -------------------------------------------------------------------------
            if (
                cv2.pointPolygonTest(roi_points_lane3, point1, False) > 0
                and cv2.pointPolygonTest(roi_points_lane2, point2, False) > 0
            ):
                cv2.rectangle(image, (int(xmin), int(ymin)), (int(xmax), int(ymax)), (0, 0, 255), 2)
                cv2.circle(image, (centroid_x, centroid_y), 5, (0, 0, 255), -1)
                car_count_lane3 += 1
        #         -------------------------------------------------------------------------
            if (
                cv2.pointPolygonTest(roi_points_lane3, point1, False) > 0
                and cv2.pointPolygonTest(roi_points_lane3, point3, False) > 0
                and cv2.pointPolygonTest(roi_points_lane2, point2, False) > 0
                and cv2.pointPolygonTest(roi_points_lane4, point4, False) > 0

            ):
                cv2.rectangle(image, (int(xmin), int(ymin)), (int(xmax), int(ymax)), (0, 0, 255), 2)
                cv2.circle(image, (centroid_x, centroid_y), 5, (0, 0, 255), -1)
                car_count_lane3 += 1
        #         --------------------------------LANE 4-----------------------------------------
            if (
                cv2.pointPolygonTest(roi_points_lane4, point2, False) > 0
                and cv2.pointPolygonTest(roi_points_lane4, point3, False) > 0
#                 and cv2.pointPolygonTest(roi_points_lane5, point2, False) > 0
#                 and cv2.pointPolygonTest(roi_points_lane5, point3, False) > 0
            ):
                cv2.rectangle(image, (int(xmin), int(ymin)), (int(xmax), int(ymax)), (255, 255, 0), 2)
                cv2.circle(image, (centroid_x, centroid_y), 5, (255, 255, 0), -1)
                car_count_lane4 += 1
        #         -------------------------------------------------------------------------
            if (
                cv2.pointPolygonTest(roi_points_lane4u, point1, False) > 0
                and cv2.pointPolygonTest(roi_points_lane4u, point4, False) > 0
            ):
                cv2.rectangle(image, (int(xmin), int(ymin)), (int(xmax), int(ymax)), (255, 255, 0), 2)
                cv2.circle(image, (centroid_x, centroid_y), 5, (255, 255, 0), -1)
                car_count_lane4 += 1
        #         -------------------------------------------------------------------------
            if (
                cv2.pointPolygonTest(roi_points_lane4upup, point1, False) > 0
                and cv2.pointPolygonTest(roi_points_lane4upup, point4, False) > 0
            ):
                cv2.rectangle(image, (int(xmin), int(ymin)), (int(xmax), int(ymax)), (255, 255, 0), 2)
                cv2.circle(image, (centroid_x, centroid_y), 5, (255, 255, 0), -1)
                car_count_lane4 += 1
        #         -------------------------------------------------------------------------
            if (
                cv2.pointPolygonTest(roi_points_lane4upup, point1, False) > 0
                and cv2.pointPolygonTest(roi_points_lane4upup, point4, False) > 0
                and cv2.pointPolygonTest(roi_points_lane6, point3, False) > 0
            ):
                cv2.rectangle(image, (int(xmin), int(ymin)), (int(xmax), int(ymax)), (255, 255, 0), 2)
                cv2.circle(image, (centroid_x, centroid_y), 5, (255, 255, 0), -1)
                car_count_lane4 += 1
        #         -------------------------------LANE 5------------------------------------------
            if (
                cv2.pointPolygonTest(roi_points_lane5, point2, False) > 0
                and cv2.pointPolygonTest(roi_points_lane5, point3, False) > 0
            ):
                cv2.rectangle(image, (int(xmin), int(ymin)), (int(xmax), int(ymax)), (0, 255, 255), 2)
                cv2.circle(image, (centroid_x, centroid_y), 5, (0, 255, 255), -1)
                car_count_lane5 += 1
 #         -----------------------------------------------------------------------
            if (
                cv2.pointPolygonTest(roi_points_lane4u, point1, False) > 0
                and cv2.pointPolygonTest(roi_points_lane4u, point4, False) > 0
                and cv2.pointPolygonTest(roi_points_lane6, point3, False) > 0
            ):
                cv2.rectangle(image, (int(xmin), int(ymin)), (int(xmax), int(ymax)), (0, 255, 255), 2)
                cv2.circle(image, (centroid_x, centroid_y), 5, (0, 255, 255), -1)
                car_count_lane5 += 1
        #         ------------------------------LANE 6-------------------------------------------
            if (
                cv2.pointPolygonTest(roi_points_lane6, point2, False) > 0
                and cv2.pointPolygonTest(roi_points_lane6, point3, False) > 0
            ):
                cv2.rectangle(image, (int(xmin), int(ymin)), (int(xmax), int(ymax)), (255, 0, 255), 2)
                cv2.circle(image, (centroid_x, centroid_y), 5, (255, 0, 255), -1)
                car_count_lane6 += 1
#         -------------------------------------------------------------------------
            if (
                cv2.pointPolygonTest(roi_points_lane5, point2, False) > 0
                and cv2.pointPolygonTest(roi_points_lane6, point3, False) > 0
            ):
                cv2.rectangle(image, (int(xmin), int(ymin)), (int(xmax), int(ymax)), (255, 0, 255), 2)
                cv2.circle(image, (centroid_x, centroid_y), 5, (255, 0, 255), -1)
                car_count_lane6 += 1
                #         -------------------------------------------------------------------------
            if (
                cv2.pointPolygonTest(roi_points_lane6, point3, False) > 0
            ):
                cv2.rectangle(image, (int(xmin), int(ymin)), (int(xmax), int(ymax)), (255, 0, 255), 2)
                cv2.circle(image, (centroid_x, centroid_y), 5, (255, 0, 255), -1)
                car_count_lane6 += 1
        #         -----------------------------LANE 7--------------------------------------------
            if (
                cv2.pointPolygonTest(roi_points_lane7, point4, False) > 0
                and cv2.pointPolygonTest(roi_points_lane7, point3, False) > 0
            ):
                cv2.rectangle(image, (int(xmin), int(ymin)), (int(xmax), int(ymax)), (255, 255, 255), 2)
                cv2.circle(image, (centroid_x, centroid_y), 5, (255, 255, 255), -1)
                car_count_lane7 += 1
        #         ------------------------LANE 8-------------------------------------------------
            if (
                cv2.pointPolygonTest(roi_points_lane8, point1, False) > 0
                and cv2.pointPolygonTest(roi_points_lane8, point2, False) > 0
            ):
                cv2.rectangle(image, (int(xmin), int(ymin)), (int(xmax), int(ymax)), (128, 0, 128), 2)
                cv2.circle(image, (centroid_x, centroid_y), 5, (128, 0, 128), -1)
                car_count_lane8 += 1
         #         -------------------------------------------------------------------------
            if (
                cv2.pointPolygonTest(roi_points_lane8, point1, False) > 0
                and cv2.pointPolygonTest(roi_points_lane8, point3, False) > 0
            ):
                cv2.rectangle(image, (int(xmin), int(ymin)), (int(xmax), int(ymax)), (128, 0, 128), 2)
                cv2.circle(image, (centroid_x, centroid_y), 5, (128, 0, 128), -1)
                car_count_lane8 += 1
        #         -------------------------------------------------------------------------
            if (
                cv2.pointPolygonTest(roi_points_lane8, point2, False) > 0
                and cv2.pointPolygonTest(roi_points_lane8, point3, False) > 0
            ):
                cv2.rectangle(image, (int(xmin), int(ymin)), (int(xmax), int(ymax)), (128, 0, 128), 2)
                cv2.circle(image, (centroid_x, centroid_y), 5, (128, 0, 128), -1)
                car_count_lane8 += 1
         #         -------------------------------------------------------------------------
            if (
                cv2.pointPolygonTest(roi_points_lane9, point2, False) > 0
                and cv2.pointPolygonTest(roi_points_lane8, point3, False) > 0
            ):
                cv2.rectangle(image, (int(xmin), int(ymin)), (int(xmax), int(ymax)), (128, 0, 128), 2)
                cv2.circle(image, (centroid_x, centroid_y), 5, (128, 0, 128), -1)
                car_count_lane8 += 1
         #         -------------------------------------------------------------------------
            if (
                cv2.pointPolygonTest(roi_points_lane8, point1, False) > 0
                and cv2.pointPolygonTest(roi_points_lane9, point2, False) > 0
            ):
                cv2.rectangle(image, (int(xmin), int(ymin)), (int(xmax), int(ymax)), (128, 0, 128), 2)
                cv2.circle(image, (centroid_x, centroid_y), 5, (128, 0, 128), -1)
                car_count_lane8 += 1
         #         -------------------------------------------------------------------------
            if (
                cv2.pointPolygonTest(roi_points_lane8, point1, False) > 0
                and cv2.pointPolygonTest(roi_points_lane9, point2, False) > 0
                and cv2.pointPolygonTest(roi_points_lane7, point3, False) > 0
                and cv2.pointPolygonTest(roi_points_lane7, point4, False) > 0
            ):
                cv2.rectangle(image, (int(xmin), int(ymin)), (int(xmax), int(ymax)), (128, 0, 128), 2)
                cv2.circle(image, (centroid_x, centroid_y), 5, (128, 0, 128), -1)
                car_count_lane8 += 1
         #         -------------------------------------------------------------------------
            if (
                cv2.pointPolygonTest(roi_points_lane8, point1, False) > 0
                and cv2.pointPolygonTest(roi_points_lane8, point2, False) > 0
                and cv2.pointPolygonTest(roi_points_lane8, point3, False) > 0
                and cv2.pointPolygonTest(roi_points_lane7, point4, False) > 0
            ):
                cv2.rectangle(image, (int(xmin), int(ymin)), (int(xmax), int(ymax)), (128, 0, 128), 2)
                cv2.circle(image, (centroid_x, centroid_y), 5, (128, 0, 128), -1)
                car_count_lane8 += 1
        #         -------------------------------------------------------------------------
            if (
                cv2.pointPolygonTest(roi_points_lane9, point1, False) > 0
                and cv2.pointPolygonTest(roi_points_lane9, point2, False) > 0
                and cv2.pointPolygonTest(roi_points_lane8, point3, False) > 0
                and cv2.pointPolygonTest(roi_points_lane8, point4, False) > 0
            ):
                cv2.rectangle(image, (int(xmin), int(ymin)), (int(xmax), int(ymax)), (128, 0, 128), 2)
                cv2.circle(image, (centroid_x, centroid_y), 5, (128, 0, 128), -1)
                car_count_lane8 += 1
        #         -------------------------------------------------------------------------
            if (
                cv2.pointPolygonTest(roi_points_lane9, point1, False) > 0
                and cv2.pointPolygonTest(roi_points_lane9, point2, False) > 0
                and cv2.pointPolygonTest(roi_points_lane7, point3, False) > 0
                and cv2.pointPolygonTest(roi_points_lane7, point4, False) > 0
            ):
                cv2.rectangle(image, (int(xmin), int(ymin)), (int(xmax), int(ymax)), (128, 0, 128), 2)
                cv2.circle(image, (centroid_x, centroid_y), 5, (128, 0, 128), -1)
                car_count_lane8 += 1
        #         -------------------------------------------------------------------------
            if (
                cv2.pointPolygonTest(roi_points_lane8, point1, False) > 0
                and cv2.pointPolygonTest(roi_points_lane8, point2, False) > 0
                and cv2.pointPolygonTest(roi_points_lane8, point3, False) > 0
                and cv2.pointPolygonTest(roi_points_lane7, point4, False) > 0
            ):
                cv2.rectangle(image, (int(xmin), int(ymin)), (int(xmax), int(ymax)), (128, 0, 128), 2)
                cv2.circle(image, (centroid_x, centroid_y), 5, (128, 0, 128), -1)
                car_count_lane8 += 1
        #         -------------------------------------------------------------------------
            if (
                cv2.pointPolygonTest(roi_points_lane8, point1, False) > 0
                and cv2.pointPolygonTest(roi_points_lane9, point2, False) > 0
                and cv2.pointPolygonTest(roi_points_lane7, point4, False) > 0
            ):
                cv2.rectangle(image, (int(xmin), int(ymin)), (int(xmax), int(ymax)), (128, 0, 128), 2)
                cv2.circle(image, (centroid_x, centroid_y), 5, (128, 0, 128), -1)
                car_count_lane8 += 1
        #         ----------------------LANE 9---------------------------------------------------
            if (
                cv2.pointPolygonTest(roi_points_lane9, point1, False) > 0
                and cv2.pointPolygonTest(roi_points_lane9, point2, False) > 0
            ):
                cv2.rectangle(image, (int(xmin), int(ymin)), (int(xmax), int(ymax)), (0, 0, 0), 2)
                cv2.circle(image, (centroid_x, centroid_y), 5, (0, 0, 0), -1)
                car_count_lane9 += 1

        # Store the result in the predicted_lanes dictionary    
        if car_count_lane1 >= 1:
            result = 1
        else:
            result = 0 
        predicted_lanes[i+1].append(result)
        if car_count_lane2 >= 1:
            result = 1
        else:
            result = 0
        predicted_lanes[i+1].append(result)
        if car_count_lane3 >= 1:
            result = 1
        else:
            result = 0 
        predicted_lanes[i+1].append(result)
        if car_count_lane4 >= 1:
            result = 1
        else:
            result = 0 
        predicted_lanes[i+1].append(result)
        if car_count_lane5 >= 1:
            result = 1
        else:
            result = 0 
        predicted_lanes[i+1].append(result)    
        if car_count_lane6 >= 1:
            result = 1
        else:
            result = 0 
        predicted_lanes[i+1].append(result)
        if car_count_lane7 >= 1:
            result = 1
        else:
            result = 0 
        predicted_lanes[i+1].append(result)
        if car_count_lane8 >= 1:
            result = 1
        else:
            result = 0 
        predicted_lanes[i+1].append(result)
        if car_count_lane9 >= 1:
            result = 1
        else:
            result = 0
        predicted_lanes[i+1].append(result)

        # Define the colors for each lane
        lane_colors = [
            (0, 255, 0),      # Green color for Lane 1
            (255, 0, 0),      # Blue color for Lane 2
            (0, 0, 255),      # Red color for Lane 3
            (255, 255, 0),    # Cyan color for Lane 4
            (0, 255, 255),    # ....
            (255, 0, 255),    # ....
            (255, 255, 255),  # ....
            (128, 0, 128),    # ....
            (0, 0, 0)         # ....
        ]

#         # Draw lanes boundaries
#         cv2.polylines(image, [roi_points_lane1], False, lane_colors[0], 1)
#         cv2.polylines(image, [roi_points_lane2], False, lane_colors[1], 1)
#         cv2.polylines(image, [roi_points_lane3], False, lane_colors[2], 1)
#         cv2.polylines(image, [roi_points_lane4], False, lane_colors[3], 1)
#         cv2.polylines(image, [roi_points_lane5], False, lane_colors[4], 1)
#         cv2.polylines(image, [roi_points_lane6], False, lane_colors[5], 1)
#         cv2.polylines(image, [roi_points_lane7], False, lane_colors[6], 1)
#         cv2.polylines(image, [roi_points_lane8], False, lane_colors[7], 1)
#         cv2.polylines(image, [roi_points_lane9], False, lane_colors[8], 1)

            
        # Display the color for each lane
        cv2.putText(image, f"Lane 1", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        cv2.putText(image, f"Lane 2", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)
        cv2.putText(image, f"Lane 3", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        cv2.putText(image, f"Lane 4", (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2)
        cv2.putText(image, f"Lane 5", (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
        cv2.putText(image, f"Lane 6", (10, 180), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 255), 2)
        cv2.putText(image, f"Lane 7", (10, 210), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        cv2.putText(image, f"Lane 8", (10, 240), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (128, 0, 128), 2)
        cv2.putText(image, f"Lane 9", (10, 270), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)        
        
        # Create the results file if it doesn't exist and write the results to a new text file

        if not os.path.exists(results_paths):
            os.makedirs(results_paths)
#         original_txt_filename = os.path.basename(txt_paths[i])
#         predicted_txt_filename = os.path.splitext(original_txt_filename)[0] + "_predicted.txt"
#         predicted_txt_path = os.path.join(results_paths, predicted_txt_filename)

        original_txt_filename = os.path.basename(txt_paths[i])
        original_txt_filename_without_query = original_txt_filename.replace("query", "")
        predicted_txt_filename = os.path.splitext(original_txt_filename_without_query)[0] + "predicted.txt"
        predicted_txt_path = os.path.join(results_paths, predicted_txt_filename)

        with open(predicted_txt_path, "w") as f:
            f.write(f"{number_of_lanes[i]}\n")
            for lane_id in lanes_ids[i]:
                if lane_id == 1:
                    f.write(f"{lane_id} {predicted_lanes[i+1][0]}\n")
                if lane_id == 2:
                    f.write(f"{lane_id} {predicted_lanes[i+1][1]}\n")
                if lane_id == 3:
                    f.write(f"{lane_id} {predicted_lanes[i+1][2]}\n")
                if lane_id == 4:
                    f.write(f"{lane_id} {predicted_lanes[i+1][3]}\n")
                if lane_id == 5:
                    f.write(f"{lane_id} {predicted_lanes[i+1][4]}\n")
                if lane_id == 6:
                    f.write(f"{lane_id} {predicted_lanes[i+1][5]}\n")                
                if lane_id == 7:
                    f.write(f"{lane_id} {predicted_lanes[i+1][6]}\n")
                if lane_id == 8:
                    f.write(f"{lane_id} {predicted_lanes[i+1][7]}\n")
                if lane_id == 9:
                    f.write(f"{lane_id} {predicted_lanes[i+1][8]}\n")
         
        print(f"Saved results for image {i+1} to predicted text path")
        print(f"image {i+1} lanes occupancy:")
        print(predicted_lanes[i+1])
        
        # Resize the image for display
        resized_image = cv2.resize(image, (1500, 800))
        # Display the image with detection results
        cv2.imshow(f"Image {i+1}", resized_image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

In [5]:
# Load and process each image in the folder
images = []
for image_path in images_paths:
    image = cv2.imread(image_path)
    images.append(image)

# Call the compare_lanes function
detect_lanes(images, number_of_lanes, lanes_ids, txt_paths, results_paths)

Detecting image 1/50



0: 320x640 2 cars, 2096.7ms
Speed: 9.1ms preprocess, 2096.7ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)


Saved results for image 1 to predicted text path
image 1 lanes occupancy:
[0, 0, 0, 0, 0, 0, 0, 1, 0]


Detecting image 2/50


0: 320x640 (no detections), 1222.2ms
Speed: 5.8ms preprocess, 1222.2ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)


...

0: 320x640 4 cars, 2069.3ms
Speed: 9.2ms preprocess, 2069.3ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)


Saved results for image 49 to predicted text path
image 49 lanes occupancy:
[0, 0, 0, 0, 1, 1, 0, 1, 0]


Detecting image 50/50


0: 320x640 4 cars, 1315.8ms
Speed: 12.2ms preprocess, 1315.8ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)


Saved results for image 50 to predicted text path
image 50 lanes occupancy:
[0, 0, 0, 0, 1, 1, 0, 1, 0]


In [ ]:
# Convert a text file to CSV in memory
def convert_to_csv(file_path):
    with open(file_path, 'r') as text_file:
        csv_data = io.StringIO()
        writer = csv.writer(csv_data)
        for line in text_file:
            writer.writerow([line.strip()])
        csv_data.seek(0)  # Reset the stream position to the beginning
        return csv_data

# Compare two CSV files and return True if they match
def compare_csv_files(file1_data, file2_data):
    file1_reader = csv.reader(file1_data)
    file2_reader = csv.reader(file2_data)
    return all(row1 == row2 for row1, row2 in zip(file1_reader, file2_reader))

# Compare all files in the folders
def compare_folders(folder1_path, folder2_path):
    # Get the list of files in each folder
    files1 = os.listdir(results_paths)
    files2 = os.listdir(ground_truth_paths)
    
    # Check if the number of files is the same
    if len(files1) != len(files2):
        print("The number of files in the folders is different.")
        return
    
    num_matched_files = 0
    num_unmatched_files = 0
    unmatched_files = []  # List to store the names of unmatched files

    # Compare each file in the folders
    for file1, file2 in zip(files1, files2):
        file1_path = os.path.join(results_paths, file1)
        file2_path = os.path.join(ground_truth_paths, file2)
        
        # Convert files to CSV in memory
        file1_data = convert_to_csv(file1_path)
        file2_data = convert_to_csv(file2_path)
        
        # Compare CSV files
        if compare_csv_files(file1_data, file2_data):
            num_matched_files += 1
        else:
            num_unmatched_files += 1
            unmatched_files.append(file1)  # Add the unmatched file name to the list
    
    # Print the number of matched and unmatched files
    print("Number of matched files:", num_matched_files)
    print("Number of unmatched files:", num_unmatched_files)
    
    # Print the names of unmatched files
    print("Unmatched files:", unmatched_files)
    
    # Calculate the score
    score = num_matched_files * 0.04
    print("Score:", score)

# Call the function to compare the folders
compare_folders(results_paths, ground_truth_paths)